In [1]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'\\CHCFPP01\Guest\MRajabi\Python Scripts'
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import scipy
from matplotlib.gridspec import GridSpec
from mailmerge import MailMerge
import shutil
import openpyxl
from scipy.optimize import curve_fit
from time import gmtime, strftime
from ipywidgets import FloatProgress
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-12-26 09:11:20


In [2]:
Project_Dir = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\IL_Crash_Assignment'
SourceDir = Project_Dir + '\\IRIS_Original'
OutputDir = Project_Dir + '\\IRIS_Cleaned'
GDB = Project_Dir + '\\CrashData_Cleaned\\IL_SafetyDB.gdb'
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-12-26 09:11:20


In [3]:
# Read IRIS data condensed
print('[{}] reading data:'.format(strftime("%Y-%m-%d %H:%M:%S")))
IRIS_DF = pd.read_csv(OutputDir + '\\IRIS_05_17.csv',low_memory=False)
print('[{}]  - {}:'.format(strftime("%Y-%m-%d %H:%M:%S"),IRIS_DF.shape))
#print('[{}] indexing ...'.format(strftime("%Y-%m-%d %H:%M:%S")))
#idx1 = pd.Index(IRIS_DF.INVENTORY)
#idx2 = pd.IntervalIndex.from_tuples([(bmp,emp) for bmp,emp in zip(IRIS_DF.BMP,IRIS_DF.EMP)],closed = 'left')
#idx3 = pd.IntervalIndex.from_tuples([(datetime(year,1,1),datetime(year+1,1,1)) for year in IRIS_DF.YEAR],closed='left')
#IRIS_DF.index = pd.MultiIndex.from_arrays([idx1,idx2,idx3],names=['INVENTORY','Milepost','Time'])
print('[{}] done.'.format(strftime("%Y-%m-%d %H:%M:%S")))

[2018-12-26 09:11:20] reading data:
[2018-12-26 09:13:10]  - (5964369, 53):
[2018-12-26 09:13:10] done.


In [4]:
def DissolveDF(DF,RID,BMP,EMP,STIM,ETIM,DissFields):
    print('[{}] dissolving by milepost: {}'.format(strftime("%Y-%m-%d %H:%M:%S"),DissFields))
    print('[{}] sorting and indexing:'.format(strftime("%Y-%m-%d %H:%M:%S")))
    F1 = [RID,STIM,ETIM,BMP,EMP]
    F2 = DissFields
    df = DF[F1+F2]
    df.index = range(0,df.shape[0])
    df = df.sort_values(by=['INVENTORY','STIM'])
    df = df.fillna(-1)
    print('[{}]  - {}:'.format(strftime("%Y-%m-%d %H:%M:%S"),df.shape))
    print('[{}] grouping intervals:'.format(strftime("%Y-%m-%d %H:%M:%S")))
    idx = df.groupby([RID,STIM,ETIM] + F2)[BMP].shift(-1) != df[EMP]
    df['EMP2'] = df.loc[idx, EMP]
    df['EMP2'] = df.groupby([RID,STIM,ETIM] + F2)['EMP2'].fillna(method='backfill')
    df['EMP2'] = df['EMP2'].fillna(df[EMP]) 
    print('[{}] aggregating groups:'.format(strftime("%Y-%m-%d %H:%M:%S")))
    sdf = df.groupby([RID,STIM,ETIM] + F2 + ['EMP2'], as_index=False).agg({BMP: 'first', EMP: 'last'}).drop(['EMP2'], axis=1)
    print('[{}]  - {}:'.format(strftime("%Y-%m-%d %H:%M:%S"),sdf.shape))

    print('[{}] dissolving by time:'.format(strftime("%Y-%m-%d %H:%M:%S")))
    print('[{}] sorting and indexing:'.format(strftime("%Y-%m-%d %H:%M:%S")))
    df = sdf
    df.index = range(0,df.shape[0])
    df = df.sort_values(by=['INVENTORY','BMP'])
    print('[{}] grouping intervals:'.format(strftime("%Y-%m-%d %H:%M:%S")))
    idx = df.groupby([RID,BMP,EMP] + F2)[STIM].shift(-1) != df[ETIM]
    df['EMP2'] = df.loc[idx, ETIM]
    df['EMP2'] = df.groupby([RID,BMP,EMP] + F2)['EMP2'].fillna(method='backfill')
    df['EMP2'] = df['EMP2'].fillna(df[ETIM]) 
    print('[{}] aggregating groups:'.format(strftime("%Y-%m-%d %H:%M:%S")))
    sdf = df.groupby([RID,BMP,EMP] + F2 + ['EMP2'], as_index=False).agg({STIM: 'first', ETIM: 'last'}).drop(['EMP2'], axis=1)
    print('[{}]  - {}:'.format(strftime("%Y-%m-%d %H:%M:%S"),sdf.shape))
    print('[{}] sorting the output:'.format(strftime("%Y-%m-%d %H:%M:%S")))
    sdf = sdf.replace(-1,np.NaN)
    sdf = sdf[F1+F2].sort_values(F1)
    print('[{}] done!'.format(strftime("%Y-%m-%d %H:%M:%S")))
    return(sdf)

In [117]:
IRIS_DF['STIM'] = IRIS_DF.YEAR.apply(lambda x:datetime(x,1,1))
IRIS_DF['ETIM'] = IRIS_DF.YEAR.apply(lambda x:datetime(x+1,1,1))
FGroups = [['MARKED_RT','MARKED_RT2','MARKED_RT3','MARKED_RT4','MRK_RT_TYP'],
           ['ROAD_NAME','FUNC_CLASS','JUR_TYPE','URBAN','OP_1_2_WAY'],
           ['SP_LIM','ACC_CNTL','TOLL','TRK_RT','PG','LANES'],
           ['AADT','AADT_YR','HCV','SU_VOL','MU_VOL','HCV_MU_YR'],
           ['LANES','LN_WTH','LN_SPC','LN_SPC_TYP','LN_SPC_WTH','MED_TYP','MED_WTH','O_SHD1_TYP','O_SHD1_WTH','O_SHD2_TYP','O_SHD2_WTH','I_SHD1_TYP','I_SHD1_WTH','I_SHD2_TYP','I_SHD2_WTH','SURF_WTH'],
           ['SURF_TYP','SURF_YR'],
           ['PRK_LT','PRK_RT']]
for f,out in zip(FGroups,['Linkage','Volume','Geometry','Pavement','Roadside']):
    df = DissolveDF(IRIS_DF,'INVENTORY','BMP','EMP','STIM','ETIM',f)
    df.to_csv(OutputDir + '\\IRIS_Diss_{}.csv'.format(out),index=False)

Linkage
[2018-12-21 15:20:54] dissolving by milepost: ['MARKED_RT', 'MARKED_RT2', 'MARKED_RT3', 'MARKED_RT4', 'MRK_RT_TYP', 'ROAD_NAME', 'FUNC_CLASS', 'JUR_TYPE', 'URBAN', 'OP_1_2_WAY', 'SP_LIM', 'ACC_CNTL', 'TOLL', 'TRK_RT', 'PG']
[2018-12-21 15:20:54] sorting and indexing:
[2018-12-21 15:21:04]  - (5964369, 20):
[2018-12-21 15:21:04] grouping intervals:
[2018-12-21 15:37:13] aggregating groups:
[2018-12-21 15:37:29]  - (2675094, 20):
[2018-12-21 15:37:29] dissolving by time:
[2018-12-21 15:37:29] sorting and indexing:
[2018-12-21 15:37:30] grouping intervals:
[2018-12-21 15:48:49] aggregating groups:
[2018-12-21 15:48:54]  - (853991, 20):
[2018-12-21 15:48:54] sorting the output:
[2018-12-21 15:49:10] done!
Volume
[2018-12-21 15:49:53] dissolving by milepost: ['AADT', 'AADT_YR', 'HCV', 'SU_VOL', 'MU_VOL', 'HCV_MU_YR']
[2018-12-21 15:49:53] sorting and indexing:
[2018-12-21 15:49:58]  - (5964369, 11):
[2018-12-21 15:49:58] grouping intervals:
[2018-12-21 16:07:41] aggregating groups:


In [6]:
FGroups = [['MARKED_RT','MARKED_RT2','MARKED_RT3','MARKED_RT4','MRK_RT_TYP','ROAD_NAME','FUNC_CLASS','JUR_TYPE','URBAN','OP_1_2_WAY','SP_LIM','ACC_CNTL','TOLL','TRK_RT','PG'],
           ['AADT','AADT_YR','HCV','SU_VOL','MU_VOL','HCV_MU_YR'],
           ['LANES','LN_WTH','LN_SPC','LN_SPC_TYP','LN_SPC_WTH','MED_TYP','MED_WTH','O_SHD1_TYP','O_SHD1_WTH','O_SHD2_TYP','O_SHD2_WTH','I_SHD1_TYP','I_SHD1_WTH','I_SHD2_TYP','I_SHD2_WTH','SURF_WTH'],
           ['SURF_TYP','SURF_YR'],
           ['PRK_LT','PRK_RT']]
for f,out in zip(FGroups,['Linkage','Volume','Geometry','Pavement','Roadside']):
    print(OutputDir + '\\IRIS_Diss_{}.csv'.format(out))
    #df = pd.read_csv(OutputDir + '\\IRIS_Diss_{}.csv'.format(out),index=False)

\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\IL_Crash_Assignment\IRIS_Cleaned\IRIS_Diss_Linkage.csv
\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\IL_Crash_Assignment\IRIS_Cleaned\IRIS_Diss_Volume.csv
\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\IL_Crash_Assignment\IRIS_Cleaned\IRIS_Diss_Geometry.csv
\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\IL_Crash_Assignment\IRIS_Cleaned\IRIS_Diss_Pavement.csv
\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\IL_Crash_Assignment\IRIS_Cleaned\IRIS_Diss_Roadside.csv


In [8]:
df = pd.read_csv(r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\IL_Crash_Assignment\IRIS_Cleaned\IRIS_Diss_Linkage.csv')

C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
df

,INVENTORY,STIM,ETIM,BMP,EMP,MARKED_RT,MARKED_RT2,MARKED_RT3,MARKED_RT4,MRK_RT_TYP,ROAD_NAME,FUNC_CLASS,JUR_TYPE,URBAN,OP_1_2_WAY,SP_LIM,ACC_CNTL,TOLL,TRK_RT,PG
0,001 00001 000835,2013-01-01,2014-01-01,0.00,0.04,NaN,NaN,NaN,NaN,NaN,CANNONBALL DR W,7.0,4.0,0,2.0,NaN,0,0,0,L1
1,001 00001 000835,2014-01-01,2015-01-01,0.00,0.04,NaN,NaN,NaN,NaN,NaN,Cannonball Dr,7.0,4.0,0,2.0,NaN,0,0,0,L1
2,001 00001 001125,2013-01-01,2017-01-01,0.00,0.06,NaN,NaN,NaN,NaN,NaN,RIVER HILLS DR,7.0,4.0,0,2.0,NaN,0,0,0,L1
3,001 00001 002250,2013-01-01,2014-01-01,0.00,0.11,NaN,NaN,NaN,NaN,NaN,W PARK ST,7.0,4.0,0,2.0,NaN,0,0,0,L1
4,001 00001 002250,2014-01-01,2015-01-01,0.00,0.11,NaN,NaN,NaN,NaN,NaN,NaN,7.0,4.0,0,2.0,NaN,0,0,0,L1
5,001 00001 003135,2013-01-01,2014-01-01,0.00,0.07,NaN,NaN,NaN,NaN,NaN,E CENTER ST,7.0,4.0,0,2.0,NaN,0,0,0,L1
6,001 00001 003135,2014-01-01,2018-01-01,0.00,0.07,NaN,NaN,NaN,NaN,NaN,Center St,7.0,4.0,0,2.0,NaN,0,0,0,L1
7,001 00001 003345,2013-01-01,2015-01-01,0.00,0.13,NaN,NaN,NaN,NaN,NaN,SHADY RD,7.0,4.0,0,2.0,NaN,0,0,0,L1
8,001 00001 003670,2013-01-01,2015-01-01,0.00,0.20,NaN,NaN,NaN,NaN,NaN,WITTLER DR,7.0,4.0,0,2.0,NaN,0,0,0,L1
9,001 00001 004565,2013-01-01,2014-01-01,0.00,0.18,NaN,NaN,NaN,NaN,NaN,W SEYMOUR ST,7.0,4.0,0,2.0,NaN,0,0,0,L1


In [10]:
# Null values count
df = pd.DataFrame(index=list(IRIS_DF.columns),columns=['Nulls'])
for y in df.index:
    df.loc[y,'Nulls'] = len(IRIS_DF[y])-IRIS_DF[y].count()
df

,Nulls
INVENTORY,0
BMP,0
EMP,0
LENGTH,0
COUNTY,0
KEY_RT_SEG,5939583
KEY_RT_TYP,0
KEY_RT_NBR,0
KEY_RT_SUF,5100622
KEY_RT_APP,0


In [12]:
df.sort_values('Nulls')

,Nulls
INVENTORY,0
SURF_TYP,0
I_SHD2_TYP,0
I_SHD1_TYP,0
O_SHD2_TYP,0
O_SHD1_TYP,0
MED_TYP,0
PG,0
TRK_RT,0
TOLL,0
